In [7]:
import cv2
import numpy as np

# ============================================================
# 0. USER INPUTS
# ============================================================

# Path to your checkerboard image
image_path = r"C:\Users\Jayaram\OneDrive\Documents\Downloads\checkerboard_9x6.png"

# Inner corners of the checkerboard (columns, rows)
# For checkerboard_9x6.png we generated earlier:
pattern_size = (9, 6)   # 9 corners across, 6 corners down

# Physical size of one square (any unit: cm, mm, etc.)
square_size = 1.0


# ============================================================
# 1. Load image & detect chessboard corners
# ============================================================

img = cv2.imread(image_path)
if img is None:
    raise FileNotFoundError(f"Could not load image from {image_path}")

gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
h, w = gray.shape[:2]
print(f"Loaded image size: {w} x {h}")

# Find chessboard corners
found, corners = cv2.findChessboardCorners(
    gray,
    patternSize=pattern_size,
    flags=cv2.CALIB_CB_ADAPTIVE_THRESH + cv2.CALIB_CB_NORMALIZE_IMAGE
)

if not found:
    raise RuntimeError(
        f"Chessboard corners not found with pattern_size={pattern_size}. "
        "Check that the image shows the full 9x6 checkerboard clearly."
    )

# Refine corner locations to sub-pixel accuracy
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 1e-4)
corners_subpix = cv2.cornerSubPix(gray, corners, (11, 11), (-1, -1), criteria)

print("Number of detected corners:", len(corners_subpix))

# Draw detected corners for visualization
img_corners = img.copy()
cv2.drawChessboardCorners(img_corners, pattern_size, corners_subpix, found)
cv2.imwrite("detected_corners.png", img_corners)
print("Detected corners image saved as detected_corners.png")


# ============================================================
# 2. Prepare 3D object points for the grid plane (Z = 0)
# ============================================================

cols, rows = pattern_size  # cols = 9, rows = 6 here

# Prepare grid like:
# (0,0,0), (1,0,0), ..., (8,0,0),
# (0,1,0), (1,1,0), ...
objp = np.zeros((cols * rows, 3), np.float32)
objp[:, :2] = np.mgrid[0:cols, 0:rows].T.reshape(-1, 2)
objp *= square_size  # scale by physical square size

# OpenCV expects lists (for multiple images)
objpoints_list = [objp]
imgpoints_list = [corners_subpix]


# ============================================================
# 3. RANSAC to remove outliers via homography
# ============================================================

# Convert to 2D (X, Y) and (u, v)
objp_2d = objp[:, :2].astype(np.float32)
imgp_2d = corners_subpix.reshape(-1, 2).astype(np.float32)

H, mask = cv2.findHomography(objp_2d, imgp_2d, cv2.RANSAC, ransacReprojThreshold=3.0)
if H is None:
    raise RuntimeError("Homography estimation failed. RANSAC could not find a good model.")

mask = mask.ravel().astype(bool)
inlier_objp = objp[mask]
inlier_imgp = corners_subpix[mask]

print(f"RANSAC inliers: {np.sum(mask)} / {len(mask)}")

# Use only RANSAC inliers for calibration
objpoints_list = [inlier_objp]
imgpoints_list = [inlier_imgp]


# ============================================================
# 4. Initial camera matrix guess
# ============================================================

fx_init = fy_init = float(max(h, w))  # rough focal length guess
cx_init = w / 2.0                     # principal point x
cy_init = h / 2.0                     # principal point y

camera_matrix_init = np.array([[fx_init, 0,       cx_init],
                               [0,       fy_init, cy_init],
                               [0,       0,       1      ]],
                              dtype=np.float64)

# Distortion: [k1, k2, p1, p2, k3], start from zeros
dist_coeffs_init = np.zeros((5, 1), dtype=np.float64)

# Flags: estimate only k1, k2 (radial), fix tangential and higher-order
flags = (
    cv2.CALIB_USE_INTRINSIC_GUESS +
    cv2.CALIB_ZERO_TANGENT_DIST +
    cv2.CALIB_FIX_K3 +
    cv2.CALIB_FIX_K4 +
    cv2.CALIB_FIX_K5 +
    cv2.CALIB_FIX_K6
)


# ============================================================
# 5. Calibrate camera (estimate intrinsics + radial distortion)
# ============================================================

rms, camera_matrix, dist_coeffs, rvecs, tvecs = cv2.calibrateCamera(
    objpoints_list,
    imgpoints_list,
    (w, h),
    camera_matrix_init,
    dist_coeffs_init,
    flags=flags
)

print("\n=== Calibration Results ===")
print("RMS reprojection error (OpenCV):", rms)
print("Estimated camera matrix K:\n", camera_matrix)
print("Estimated distortion coefficients [k1, k2, p1, p2, k3]:\n", dist_coeffs.ravel())

k1, k2 = dist_coeffs.ravel()[:2]
print(f"Estimated radial distortion parameters: k1={k1:.6e}, k2={k2:.6e}")


# ============================================================
# 6. Undistort the image
# ============================================================

new_camera_matrix, roi = cv2.getOptimalNewCameraMatrix(
    camera_matrix, dist_coeffs, (w, h), 1, (w, h)
)

undistorted_img = cv2.undistort(img, camera_matrix, dist_coeffs, None, new_camera_matrix)
cv2.imwrite("undistorted_image.png", undistorted_img)
print("Undistorted image saved as undistorted_image.png")


# ============================================================
# 7. Compute undistorted grid corners
# ============================================================

inlier_imgp_undist = cv2.undistortPoints(
    inlier_imgp.reshape(-1, 1, 2),
    camera_matrix,
    dist_coeffs,
    P=new_camera_matrix
)
inlier_imgp_undist = inlier_imgp_undist.reshape(-1, 2)

undist_corners_vis = undistorted_img.copy()
for pt in inlier_imgp_undist.astype(int):
    cv2.circle(undist_corners_vis, tuple(pt), 4, (0, 0, 255), -1)

cv2.imwrite("undistorted_grid_corners.png", undist_corners_vis)
print("Undistorted grid corners saved as undistorted_grid_corners.png")


# ============================================================
# 8. Reproject grid into distorted image & compute residuals
# ============================================================

rvec = rvecs[0]
tvec = tvecs[0]

proj_points, _ = cv2.projectPoints(
    inlier_objp,
    rvec,
    tvec,
    camera_matrix,
    dist_coeffs
)
proj_points = proj_points.reshape(-1, 2)
obs_points = inlier_imgp.reshape(-1, 2)

errors = np.linalg.norm(obs_points - proj_points, axis=1)

mean_error = np.mean(errors)
rmse_error = np.sqrt(np.mean(errors ** 2))
max_error = np.max(errors)

print("\n=== Reprojection Error (ours) ===")
print(f"Mean reprojection error (pixels): {mean_error:.4f}")
print(f"RMSE reprojection error (pixels): {rmse_error:.4f}")
print(f"Max reprojection error (pixels): {max_error:.4f}")

vis = img.copy()
for p_obs, p_pred in zip(obs_points.astype(int), proj_points.astype(int)):
    cv2.circle(vis, tuple(p_obs), 4, (0, 255, 0), -1)  # observed in green
    cv2.circle(vis, tuple(p_pred), 2, (0, 0, 255), -1)  # predicted in red

cv2.imwrite("reprojection_visualization.png", vis)
print("Reprojection visualization saved as reprojection_visualization.png")


Loaded image size: 800 x 560
Number of detected corners: 54
Detected corners image saved as detected_corners.png
RANSAC inliers: 54 / 54

=== Calibration Results ===
RMS reprojection error (OpenCV): 3.212756867358388e-14
Estimated camera matrix K:
 [[799.99999998   0.         400.        ]
 [  0.         799.99999998 280.        ]
 [  0.           0.           1.        ]]
Estimated distortion coefficients [k1, k2, p1, p2, k3]:
 [-2.32693993e-15  8.39780471e-15  0.00000000e+00  0.00000000e+00
  0.00000000e+00]
Estimated radial distortion parameters: k1=-2.326940e-15, k2=8.397805e-15
Undistorted image saved as undistorted_image.png
Undistorted grid corners saved as undistorted_grid_corners.png

=== Reprojection Error (ours) ===
Mean reprojection error (pixels): 0.0000
RMSE reprojection error (pixels): 0.0000
Max reprojection error (pixels): 0.0000
Reprojection visualization saved as reprojection_visualization.png
